In [6]:
# load dataset
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import simplejson as json

df = pd.read_csv("/users/akuppam/documents/Hprog/Py/rnb1015_2.csv")
df.head(5)

,date,region,marketing,visits,br,inq,gb,cb,nb,ss,ts,listings
0,1/1/16,AMR,NotPaid,1256288,6043,66022,6071,571,5500,485968.08000,144695.894200,10242216
1,1/1/16,AMR,Paid,189297,1092,11806,743,2,741,0.00000,14138.809600,2133795
2,1/1/16,UK,NotPaid,201797,435,9662,436,33,403,72698.92472,8317.105056,3561696
3,1/1/16,UK,Paid,55934,88,2749,61,1,60,0.00000,815.336426,890424
4,1/1/16,CE,NotPaid,200786,573,9250,435,31,404,40490.73159,12016.782410,3062480


In [7]:
df = pd.read_csv("/users/akuppam/documents/Hprog/Py/rnb1015_2.csv")
df = df.loc[(df['region'] == 'UK') & (df['marketing'] == 'NotPaid')]
df = df[~(df['nb'] < 0)]
df.describe()
df.index = pd.to_datetime(df.date)
cols = ["nb"]
y = df[cols]
y.head()

,nb
date,
2016-01-01,403
2016-01-02,451
2016-01-03,682
2016-01-04,672
2016-01-05,705


In [ ]:
import warnings
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	history = [x for x in X]
	# make predictions
	predictions = list()
	for t in range(len(X)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit(transparams=True)
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(X[t])
	# calculate out of sample error
	error = mean_squared_error(X, predictions)
	mape = np.mean(abs((X-predictions)/X))
	return error
	return mape

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_params = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					mse = evaluate_arima_model(dataset, order)
					if mse < best_score:
						best_score, best_params = mse, order
					print('ARIMA %s MSE=%.3f' % (order,mse))
				except:
					continue
	print('Best ARIMA %s MSE=%.3f' % (best_params, best_score))
	#model_params = [p,d,q]
	print("best order =", model_params)
	with open('model_params.json','w') as out_params:
		json.dump(best_params, out_params)

# load dataset
df.index = pd.to_datetime(df.date)
cols = ["nb"]
y = df[cols]
series = y['nb']

# evaluate parameters
p_values = range(0, 13)
d_values = range(0, 2)
q_values = range(0, 13)
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)

# select the best set of p,d,q
with open('model_params.json') as json_file:
    out_params = json.load(json_file)
out_params
p = out_params[0]
d = out_params[1]
q = out_params[2]

# apply model
model = ARIMA(series, order=(p,d,q))
model_fit = model.fit(transparams=True)
print(model_fit.summary())

# Store predictions
import numpy as np
X = series.values
history = [x for x in X]
predictions = list()
for t in range(len(X)):
	model = ARIMA(history, order=(p,d,q))
	model_fit = model.fit(disp=0)
	output = model_fit.forecast()
	yhat = output[0]
	predictions.append(yhat)
	obs = X[t]
	history.append(obs)

# Get forecast 442 steps ahead in future (10/15/2018 to 12/31/2019)
yhat = model_fit.forecast(steps=442)
series_arima = predictions.append(yhat)

ARIMA (0, 0, 0) MSE=61255.079
ARIMA (0, 0, 1) MSE=28645.326
ARIMA (0, 0, 2) MSE=21510.962
ARIMA (0, 0, 3) MSE=18114.742
ARIMA (0, 0, 4) MSE=15821.966
ARIMA (0, 0, 6) MSE=15105.721
ARIMA (0, 0, 7) MSE=14171.619
ARIMA (0, 0, 9) MSE=11946.696
ARIMA (0, 0, 10) MSE=11561.584
ARIMA (0, 0, 11) MSE=11248.866
ARIMA (0, 0, 12) MSE=11245.856
ARIMA (0, 1, 0) MSE=14788.145
ARIMA (0, 1, 1) MSE=13881.918
ARIMA (0, 1, 2) MSE=12546.026
ARIMA (0, 1, 3) MSE=12529.379
ARIMA (1, 0, 0) MSE=13930.255
ARIMA (1, 0, 1) MSE=13602.828
